In [7]:
#import
import pandas as pd
import numpy as np
from pathlib import Path
import os
import matplotlib.pyplot as plt
from scipy import signal

In [2]:
#export
TIMESLICES=16
SCANS=404
FILENAME = "task-rest_acq-1400_bold_full.1D"

In [3]:
#export
def getids(filepath):
    files = list(Path(filepath).glob("*1D"))
    ids = set()
    for file in files:
        imid = "_".join(str(file).split("_")[1:3])
        ids.add(imid)
    return ids, files

In [4]:
#export
def reshape_stacks(ids, files):
    for imid in ids:
        stack = [stack_entry for stack_entry in files if imid in str(stack_entry)]
        stack = sorted(stack)
        assert len(stack) == TIMESLICES
        full_stack = [np.genfromtxt(stack_entry) for stack_entry in stack]
        stack_array = np.array(full_stack)
        try:
            stack_reordered = np.reshape(stack_array, (TIMESLICES*SCANS, 6), order="F")
        except ValueError as e:
            print(e)
            continue
        assert (stack_reordered[16, :] == full_stack[0][1, :]).all
        np.savetxt("{}_{}".format(imid, FILENAME), stack_reordered, fmt="%.04f")

In [5]:
#fire run
def run(filepath):
    filepath = Path(filepath)
    ids, files = getids(filepath)
    reshape_stacks(ids, files)


cannot reshape array of size 7776 into shape (6464,6)
cannot reshape array of size 14880 into shape (6464,6)


In [ ]:
run("../../data/stacks/")

In [ ]:
sampling_rate = 1/(1.4/16)
time = np.arange(0, 6464, 1/sampling_rate)
data = stack_reordered[:, -1]
fourier_transform = np.fft.rfft(data)
abs_fourier_transform = np.abs(fourier_transform)
power_spectrum = np.square(abs_fourier_transform)
frequency = np.linspace(0, sampling_rate/2, len(power_spectrum))
plt.plot(frequency, power_spectrum)
plt.xlim(0, 6)
plt.yscale("log")
plt.show()

In [ ]:
power_spectrum[:, ]

In [ ]:
plt.plot?